In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


## modelagem GLM

In [2]:
%%time
df_mod_mp_feb23jul24_v2 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23jul24_v2.csv",
                     encoding='latin-1',
                     delimiter='|')


df_mod_mp_feb23jul24_v2.shape

CPU times: total: 14.2 s
Wall time: 15.5 s


(575924, 105)

In [3]:
premio_gbm = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_julho_gbm_python_v2.csv",
                     encoding='latin-1',
                     delimiter='|')
premio_gbm.shape

(575924, 3)

In [4]:
df_mod_mp_feb23jul24_v2 = pd.merge(df_mod_mp_feb23jul24_v2,premio_gbm[['Chave','GBM_mercado']],on='Chave', how='left')

In [5]:
#df_mod_mp_feb23jul24_v2 = pd.merge(df_mod_mp_feb23jul24_v2,premio_gbm,how='left',on=['Chave'])

In [6]:
#df.drop(''Unnamed: 0'', axis=1)

In [7]:
df_mod_mp_feb23jul24_v2['Grupo_veiculo_new']=df_mod_mp_feb23jul24_v2['novo']

In [8]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(df_mod_mp_feb23jul24_v2)

Your selected dataframe has 107 columns and 575924 Rows.
There are 16 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
perf_cond_menor26,0,391245,67.9,391245,67.9,object
perf_est_civil,0,227357,39.5,227357,39.5,object
Corr_Classif_5_hibr,0,99433,17.3,99433,17.3,object
perf_modalidade,0,40243,7.0,40243,7.0,object
novo,0,1622,0.3,1622,0.3,object
Grupo_veiculo_new,0,1622,0.3,1622,0.3,object
per_com,652,1372,0.2,2024,0.4,float64
MD_est_civil,0,566,0.1,566,0.1,object
FABRICA,0,378,0.1,378,0.1,object
cod_nacionalidade,0,378,0.1,378,0.1,float64


#### prepara base de modelagem glm

In [9]:
df_mod_filtered=df_mod_mp_feb23jul24_v2[["premio_ofertado","premio_total", 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade","per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","classe_bonus","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod','GBM_mercado']]

df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]
df_mod_filtered['GBM_mercado']  = df_mod_filtered['GBM_mercado']/(1-df_mod_filtered['per_com']/100)


In [10]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['FABRICA'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('premio_ofertado >100')
df_mod_filtered = df_mod_filtered.query('MD_idade !="ERRO"')
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total <=2.3)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total >=0.7)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.UF != "SP") | (~df_mod_filtered.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]

df_mod_filtered.shape

(363905, 51)

In [11]:
#df_mod_filtered['GBM_mercado'].describe()

In [12]:
#print(df_mod_filtered['classe_bonus'].value_counts(ascending=False))

In [13]:
df_mod=df_mod_filtered.copy()

In [14]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(object)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [15]:
df_mod = categorizar(df_mod)

In [16]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

(291023, 65)

In [17]:
df_train['Premio_IC_100']=df_train['Premio_IC100']

In [18]:
#df_cat_glm.columns.to_list()

In [19]:
#print(df_train['fator_ajuste_bin'].value_counts(ascending=False)) 

In [20]:
df_num = df_train[['premio', 'per_com', 'MD_tempo_casa_2', 'is_casco_ofer','Premio_IC_100','premio_anterior','GBM_mercado']]
df_cat = df_train[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_y = df_train[['ind_renovacao']]
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_cat_y,df_num, df_cat_glm],axis=1)

In [21]:
#df_cat_glm.columns.to_list()

In [22]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
#formula =  'ind_renovacao ~ 1 + np.log(np.divide(premio, GBM_mercado)) ' 
formula =  'ind_renovacao ~ 1 + np.log(premio) +  np.divide(premio,is_casco_ofer) + np.log(np.divide(premio, GBM_mercado)) + np.log(np.divide(premio, Premio_IC_100))+ np.log((np.divide(premio, premio_anterior))) + np.log(MD_tempo_casa_2+1) + per_com + per_com_bin_p_1_10 + canal_bin_Corretor_Mais + Cod_End_bin_13509 + fator_ajuste_bin_f_101_120 + classe_bonus_bin_00 + UF_bin_AL_CE_RS + UF_bin_AP_DF_ES_PA_PB_RN_SC + UF_bin_GO_MA_MG + UF_bin_MT_PR + UF_bin_RJ + MD_comb_bin_Gasolina + md_tipo_renov_ant_bin_1_SeguroNovo + md_tipo_renov_ant_bin_3_RenovCongenere + fx_ir_12m_bin_01_00_20 + fx_ir_12m_bin_03_40_60 + fx_ir_12m_bin_04_60_80 + fx_ir_12m_bin_05_80_100 + MD_tipo_renov_bin_3_RenovPropria_com_sin + idade_ult_RNS_bin_01_05_anos + idade_ult_RNS_bin_06_inf ' 
#formula =  'ind_renovacao ~ 1 + np.log(premio) + np.divide(premio,is_casco_ofer) + np.log(np.divide(premio, Premio_IC100))+ np.log((np.divide(premio, premio_anterior))) + np.log(MD_tempo_casa_2+1) + per_com + per_com_bin_p_1_10 + canal_bin_Corretor_Mais + Cod_End_bin_13509 + fator_ajuste_bin_f_101_120 + classe_bonus_bin_00 + UF_bin_AL_CE_RS + UF_bin_AP_DF_ES_PA_PB_RN_SC + UF_bin_GO_MA_MG + UF_bin_MT_PR + UF_bin_RJ + MD_comb_bin_Gasolina + md_tipo_renov_ant_bin_1_SeguroNovo + md_tipo_renov_ant_bin_3_RenovCongenere + fx_ir_12m_bin_01_00_20 + fx_ir_12m_bin_03_40_60 + fx_ir_12m_bin_04_60_80 + fx_ir_12m_bin_05_80_100 + MD_tipo_renov_bin_3_RenovPropria_com_sin + idade_ult_RNS_bin_01_05_anos + idade_ult_RNS_bin_06_inf ' 
model_glm = smf.logit(formula , data=df_cat_glm).fit()
model_glm.summary()

Optimization terminated successfully.
         Current function value: 0.560766
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:               291023
Model:                          Logit   Df Residuals:                   290995
Method:                           MLE   Df Model:                           27
Date:                Thu, 22 Aug 2024   Pseudo R-squ.:                  0.1882
Time:                        00:45:50   Log-Likelihood:            -1.6320e+05
converged:                       True   LL-Null:                   -2.0104e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        0.6010      0.118      5.092      0.000       0.370       0.832
np.log(premio)                                  -0.1513      0.014    -10.787      0.000      -0.179      -0.124
np.divide(premio, is_casco_ofer)                -1.9242      0.240     -8.026      0.000      -2.394      -1.454
np.log(np.divide(premio, GBM_mercado))          -0.9005      0.027    -33.026      0.000      -0.954      -0.847
np.log(np.divide(premio, Premio_IC_100))        -0.9288      0.016    -56.652      0.000      -0.961      -0.897
np.log((np.divide(premio, premio_anterior)))    -1.1300      0.021    -53.769      0.000      -1.171      -1.089
np.log(MD_tempo_casa_2 + 1)                      0.2656      0.012     21.292      0.000       0.241       0.290
per_com                                          0.0110      0.001     12.324      0.000       0.009       0.013
per_com_bin_p_1_10                               0.1572      0.026      6.132      0.000       0.107       0.207
canal_bin_Corretor_Mais                          0.0683      0.011      6.422      0.000       0.047       0.089
Cod_End_bin_13509                               -0.2266      0.015    -15.409      0.000      -0.255      -0.198
fator_ajuste_bin_f_101_120                       0.1930      0.030      6.475      0.000       0.135       0.251
classe_bonus_bin_00                             -0.2988      0.044     -6.731      0.000      -0.386      -0.212
UF_bin_AL_CE_RS                                 -0.2465      0.014    -17.533      0.000      -0.274      -0.219
UF_bin_AP_DF_ES_PA_PB_RN_SC                     -0.2347      0.014    -16.351      0.000      -0.263      -0.207
UF_bin_GO_MA_MG                                 -0.0984      0.015     -6.418      0.000      -0.128      -0.068
UF_bin_MT_PR                                    -0.1547      0.015    -10.460      0.000      -0.184      -0.126
UF_bin_RJ                                        0.1750      0.021      8.213      0.000       0.133       0.217
MD_comb_bin_Gasolina                             0.2990      0.020     14.802      0.000       0.259       0.339
md_tipo_renov_ant_bin_1_SeguroNovo              -0.4524      0.017    -25.927      0.000      -0.487      -0.418
md_tipo_renov_ant_bin_3_RenovCongenere          -0.1465      0.014    -10.767      0.000      -0.173      -0.120
fx_ir_12m_bin_01_00_20                          -0.7454      0.018    -41.468      0.000      -0.781      -0.710
fx_ir_12m_bin_03_40_60                           0.4683      0.010     45.047      0.000       0.448       0.489
fx_ir_12m_bin_04_60_80                           1.1445      0.014     81.964      0.000       1.117       1.172
fx_ir_12m_bin_05_80_100                          2.0254      0.026     79.360      0.000       1.975       2.075
MD_tipo_renov_bin_3_RenovPropria_com_sin        -0.2652      0.022    -11.931

In [23]:
#df_train["prob_est"]=model.predict(df_train)

## Base Para escorar e otimizar

In [24]:
#df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table 7 Base Case Total  - teste novo.csv",

In [25]:
df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/data table - Otimização Atuarial Segmentacao Earnix.csv",
                     encoding='latin-1',
                     delimiter='|')

df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

In [26]:
np.mean(df_lote_102024['Renewal Demand'])

0.5075687198952625

In [27]:
#df_lote_102024.groupby(['Amostra_2'])['Premio_oferecido_com_batente','Premium Parameter','Mercado_Mediana','GBM_mercado',"Renewal Demand"].describe().T

In [28]:
#df_lote_092024.columns.to_list()

In [29]:
df_lote_escorar=df_lote_102024.copy()

In [30]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [31]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [32]:
df_lote = categorizar(df_lote_escorar)

In [33]:
#df_num = df_train[['premio', 'per_com', 'MD_tempo_casa_2', 'is_casco_ofer','Premio_IC100','premio_anterior']]
#df_cat = df_train[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
#df_cat_y = df_train[['ind_renovacao']]
#df_cat_glm=pd.get_dummies(df_cat)
#df_cat_glm=pd.concat([df_lote_escorar, df_cat_glm],axis=1)

In [34]:
df_lote.shape

(1459, 137)

In [35]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [36]:
#df_cat_glm2.columns.to_list()

In [37]:
df_lote["Demanda_base"]=model_glm.predict(df_lote)
df_lote["per_com_pct"] = df_lote['per_com']/100
df_lote["premio_prob"] = df_lote["Demanda_base"]*df_lote["premio"]
df_lote["premio_anterior_prob"] = df_lote["Demanda_base"]*df_lote["premio_anterior"]
df_lote["Act_cost_prob"] = df_lote["Demanda_base"]*df_lote["Act_cost"]                                                                
df_lote["comissao_valor"] = df_lote["per_com_pct"] *df_lote["premio"]
df_lote["comissao_valor_prob"] = df_lote["per_com_pct"] *df_lote["premio_prob"]
df_lote["per_com_pct"] = df_lote['per_com']/100

In [ ]:
('TUT_Premium'-'TARIFA')/'TARIFA' (offered)	Relative Core Optimization Increase [Offered]
('TUT_Premium'-'TARIFA')/'TARIFA' (written)	Relative Core Optimization Increase [Written]
TUT_Premium'-'TUT_Previous Premium'  (offered) / renewal Demand	Core Premium Increase [Offered]
TUT_Premium'-'TUT_Previous Premium' (written)  / Renewal Demand	Core Premium Increase [Written]
Actuarial Cost Parameter	Risco Oferecido
Actuarial Cost Parameter	Risco Renovado
per_com_'/100*'Premium Parameter' / 'per_com_'/100*'Premium Parameter' (Offered)	% Comissão Oferecida
per_com_'/100*'Premium Parameter' / 'per_com_'/100*'Premium Parameter' (Written)	% Comissão Renovado
Actuarial Cost Parameter / Premium Parameter (Offered)	Loss Ratio [%] - Offered (TW)
Actuarial Cost Parameter / Premium Parameter (Written)	Loss Ratio [%] - Written (TW)
('Premium Parameter'*('per_com_'/100+'Coef_Carregamento_Produção')+'Act_cost_TW') / Premium Parameter (Offered)	Combined Ratio [%] - Offered
('Premium Parameter'*('per_com_'/100+'Coef_Carregamento_Produção')+'Act_cost_TW') / Premium Parameter (Written)	Combined Ratio [%] - Written


In [ ]:
Relative_Core_Optimization_Increase_Offered

In [ ]:
#df_lote.groupby(["Amostra"])["prob_est" , "Renewal Demand"].describe().T

In [38]:
apolice_ofertada  = np.round(df_lote.shape[0],0)
apolice_estimada  = np.round(np.sum(df_lote["Demanda_base"]),0)
premio_ofertado   = np.mean(df_lote["premio"])
retencao_estimada = np.mean(df_lote["Demanda_base"])
premio_esperado   = np.sum(df_lote["premio_prob"])/np.sum(df_lote["Demanda_base"])
risco_ofertado    = np.mean(df_lote["Act_cost"])
risco_esperado    = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["Demanda_base"])
sp_ofertado       = np.sum(df_lote["Act_cost"])/np.sum(df_lote["premio"])
sp_esperado       = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["premio_prob"])
comis_ofertado    = np.sum(df_lote["comissao_valor"])/np.sum(df_lote["premio"])
comis_esperado    = np.sum(df_lote["comissao_valor_prob"])/np.sum(df_lote["premio_prob"])



premio_anterior   = np.mean(df_lote["premio_anterior"])
premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["Demanda_base"])

ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))
print('premio_anterior: %s' % np.round(premio_anterior,2))
print('premio_anterior_w: %s' % np.round(premio_anterior_w,2))


Apolices Ofertadas: 1459
Apolices Renovadas: 732.0
premio_ofertado: 2993.3
retencao_estimada : 50.16
premio_esperado" : 2454.63
risco_ofertado" : 1726.89
risco_esperado" : 1389.03
sp_ofertado" : 57.69
sp_esperado" : 56.59
ic_ofertado" : 100.78
ic_esperado" : 99.88
premio_anterior: 2863.09
premio_anterior_w: 2623.47


In [39]:
np.mean(df_lote_102024['Renewal Demand'])

0.5075687198952625

## Otimização Teste 

In [40]:
#print(df_lote['canal'].value_counts(ascending=False))

In [41]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

(1459, 175)

In [42]:
#df_lote_otim_atu.groupby(["Amostra"])["Demanda_base"].describe().T

In [43]:
#df_lote_otim = df_lote_otim_atu.sample(n =100) 
#df_lote_otim.shape

In [44]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

(1459, 175)

In [45]:
np.mean(df_lote_otim['Renewal Demand'])

0.5075687198952625

In [46]:
np.mean(df_lote_otim['Demanda_base'])

0.5015768113336583

#########################################

In [47]:
df_relatorio_bc = df_lote_otim.copy()

df_relatorio_bc["per_com_pct"] = df_relatorio_bc['per_com']/100
df_relatorio_bc["premio_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["premio"]
df_relatorio_bc["Act_cost_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["Act_cost"]                                                                
df_relatorio_bc["comissao_valor"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio"]
df_relatorio_bc["comissao_valor_prob"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_bc.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_bc["Demanda_base"]),0)
premio_ofertado   = np.mean(df_relatorio_bc["premio"])
retencao_estimada = np.mean(df_relatorio_bc["Demanda_base"])
premio_esperado   = np.sum(df_relatorio_bc["premio_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
risco_ofertado    = np.mean(df_relatorio_bc["Act_cost"])
risco_esperado    = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
sp_ofertado       = np.sum(df_relatorio_bc["Act_cost"])/np.sum(df_relatorio_bc["premio"])
sp_esperado       = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_bc["comissao_valor"])/np.sum(df_relatorio_bc["premio"])
comis_esperado    = np.sum(df_relatorio_bc["comissao_valor_prob"])/np.sum(df_relatorio_bc["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

Apolices Ofertadas: 1459
Apolices Renovadas: 732.0
premio_ofertado: 2993.3
retencao_estimada : 50.16
premio_esperado" : 2454.63
risco_ofertado" : 1726.89
risco_esperado" : 1389.03
sp_ofertado" : 57.69
sp_esperado" : 56.59
ic_ofertado" : 100.78
ic_esperado" : 99.88


In [48]:
np.mean(df_relatorio_bc['Renewal Demand'])

0.5075687198952625

In [49]:
np.mean(df_relatorio_bc['Demanda_base'])

0.5015768113336583

In [50]:
#df2 = df[['premio','is_casco_ofer','Cod_End_bin_Demais','Act_cost']]

In [51]:
#df2.to_excel(r'analise_teste.xlsx',  header='true')

In [52]:
#v2=np.mean(df2['is_casco_ofer'])
#indicadora = np.array(df2['Cod_End_bin_Demais']) 
#precos_iniciais = np.array(df2['premio']) 

In [53]:
#[x for x in df.columns if "com_bin" in x]

In [54]:
#df.chave.dtypes

In [56]:
df2 = df_lote_otim

P2=(df2['premio'])
#P2 = precos_otimizados
X1=np.array(df2['is_casco_ofer'])
X2=np.array(df2['GBM_mercado'])
X3=np.array(df2['Premio_IC_100'])
X4=np.array(df2['premio_anterior'])
X5=np.array(df2['MD_tempo_casa_2'])
X6=np.array(df2['per_com'])
X7=np.array(df2['per_com_bin_p_1_10'])
X8=np.array(df2['canal_bin_Corretor_Mais'])
X9=np.array(df2['Cod_End_bin_13509'])
X10=np.array(df2['fator_ajuste_bin_f_101_120'])
X11=np.array(df2['classe_bonus_bin_00'])
X12=np.array(df2['UF_bin_AL_CE_RS'])
X13=np.array(df2['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df2['UF_bin_GO_MA_MG'])
X15=np.array(df2['UF_bin_MT_PR'])
X16=np.array(df2['UF_bin_RJ'])
X17=np.array(df2['MD_comb_bin_Gasolina'])
X18=np.array(df2['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df2['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df2['fx_ir_12m_bin_01_00_20'])
X21=np.array(df2['fx_ir_12m_bin_03_40_60'])
X22=np.array(df2['fx_ir_12m_bin_04_60_80'])
X23=np.array(df2['fx_ir_12m_bin_05_80_100'])
X24=np.array(df2['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df2['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df2['idade_ult_RNS_bin_06_inf'])

def demanda(P2,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(0.601-0.1513 * np.log(P2)-1.9242 * np.divide(P2, X1)-0.9005 * np.log(np.divide(P2, X2))-0.9288 * np.log(np.divide(P2, X3))-1.13 * np.log((np.divide(P2, X4)))  + 0.2656 * np.log(X5 + 1)  + 0.011 * X6  + 0.1572 * X7  + 0.0683 * X8-0.2266 * X9  + 0.193 * X10-0.2988 * X11-0.2465 * X12-0.2347 * X13-0.0984 * X14-0.1547 * X15  + 0.175 * X16  + 0.299 * X17-0.4524 * X18-0.1465 * X19-0.7454 * X20  + 0.4683 * X21  + 1.1445 * X22  + 2.0254 * X23-0.2652 * X24-0.485*X25-0.43 * X26)
    return D/(1+D)

In [57]:
np.mean(P2)

2993.303855515689

In [58]:
demanda(P2,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26).mean()

0.5015731841949308

In [60]:
df = df_lote_otim

#P2=(df2['premio'])
#P2 = precos_otimizados
X1=np.array(df2['is_casco_ofer'])
X2=np.array(df2['GBM_mercado'])
X3=np.array(df2['Premio_IC_100'])
X4=np.array(df2['premio_anterior'])
X5=np.array(df2['MD_tempo_casa_2'])
X6=np.array(df2['per_com'])
X7=np.array(df2['per_com_bin_p_1_10'])
X8=np.array(df2['canal_bin_Corretor_Mais'])
X9=np.array(df2['Cod_End_bin_13509'])
X10=np.array(df2['fator_ajuste_bin_f_101_120'])
X11=np.array(df2['classe_bonus_bin_00'])
X12=np.array(df2['UF_bin_AL_CE_RS'])
X13=np.array(df2['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df2['UF_bin_GO_MA_MG'])
X15=np.array(df2['UF_bin_MT_PR'])
X16=np.array(df2['UF_bin_RJ'])
X17=np.array(df2['MD_comb_bin_Gasolina'])
X18=np.array(df2['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df2['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df2['fx_ir_12m_bin_01_00_20'])
X21=np.array(df2['fx_ir_12m_bin_03_40_60'])
X22=np.array(df2['fx_ir_12m_bin_04_60_80'])
X23=np.array(df2['fx_ir_12m_bin_05_80_100'])
X24=np.array(df2['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df2['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df2['idade_ult_RNS_bin_06_inf'])


In [67]:
%%time
######
import numpy as np
import sympy as sp
from scipy.optimize import fsolve


target_demand=0.5015731841949308


df['target_demand']=target_demand

def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(0.601-0.1513 * np.log(P)-1.9242 * np.divide(P, X1)-0.9005 * np.log(np.divide(P, X2))-0.9288 * np.log(np.divide(P, X3))-1.13 * np.log((np.divide(P, X4)))  + 0.2656 * np.log(X5 + 1)  + 0.011 * X6  + 0.1572 * X7  + 0.0683 * X8-0.2266 * X9  + 0.193 * X10-0.2988 * X11-0.2465 * X12-0.2347 * X13-0.0984 * X14-0.1547 * X15  + 0.175 * X16  + 0.299 * X17-0.4524 * X18-0.1465 * X19-0.7454 * X20  + 0.4683 * X21  + 1.1445 * X22  + 2.0254 * X23-0.2652 * X24-0.485*X25-0.43 * X26)
    return D/(1+D)


# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(P):
    
    df['P'] = df['premio']
    #df['precos']=df['premio'].values
    
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada1'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
    
    df['margem_esperada_otimizada'] =   df['retencao_otimizada1']*(df['P'] - df['Act_cost'])
    
    # Calcular a margem esperada total (somatório)
    margem_esperada_total = df['margem_esperada_otimizada'].sum() 

    # Maximizar a margem 
    return -margem_esperada_total

# Definir os limites para os preços otimizado
nome_variaveis = ['LB','UB']
limites=np.array(df.loc[:,nome_variaveis])

#nome_variaveis2 = ['retencao_otimizada1','retencao_otimizada1']
#A=np.array(df.loc[:,nome_variaveis2])

a = np.squeeze(np.asarray(target_demand))*0.999
b = np.squeeze(np.asarray(target_demand))*1.001

# restricão demanda alvo
from scipy.optimize import LinearConstraint, Bounds

lc = LinearConstraint([a,b])

# Definir as restrições no formato exigido pelo método 'minimize'
#restricoes = ({'type': 'eq', 'fun': restricao})

# Executar a otimização usando o método de Differential Evolution 
resultado = differential_evolution(objetivo, 
                                   bounds=limites, 
                                   strategy='best1bin',
                                   maxiter=3000, 
                                   popsize=11,
                                   seed=df.shape[0],
                                   constraints=lc)

# Resultados
print(resultado.message) 
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun



# Verificar os preços otimizados e a margem esperada otimizada 
#df['precos_otimizados'] = resultado.x 


ValueError: shapes (1,2) and (1459,) not aligned: 2 (dim 1) != 1459 (dim 0)

In [68]:

fator_retencao = np.mean(df['Demanda_base'])/target_demand

df['precos_otimizados1']=precos_otimizados

df['margem_esperada_total_otimizada1'] = df['retencao_otimizada1']*(df['precos_otimizados1']-df['Act_cost'])

P = precos_otimizados *1

df['precos_otimizados'] = P

df['retencao_otimizada'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

df['margem_esperada_base_case'] = df['Demanda_base']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()

# Exibir os resultados

print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada1:", df['margem_esperada_total_otimizada1'].sum())
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Retenção Média Otimizada1:", df['retencao_otimizada1'].mean())
print("Retenção Média Base Case:", df['Demanda_base'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Tamanho da Base:", df.shape[0])


Margem Esperada Total Base Case: 779807.0247068419
Margem Esperada Total Otimizada1: 864214.6088270601
Margem Esperada Total Otimizada: 787266.6275917034
Retenção Média Otimizada: 0.503565634562428
Retenção Média Otimizada1: 0.5525332890148633
Retenção Média Base Case: 0.5015768113336583
Premio Medio Otimizado: 2977.9262062420403
Premio Medio Base Case: 2993.303855515689
Tamanho da Base: 1459


In [ ]:
df[["precos_otimizados","premio"]].describe().T

In [ ]:
np.mean(df['precos_otimizados'])

In [ ]:
np.mean(P)/np.mean(df["premio"])

In [ ]:
df_t = df.query('chave == 167700061973100001')
df_t[["varia_prem_anterior","varia_prem_ofer","varia_prem_anterior_earnix","varia_prem_ofer_earnix"]]

In [ ]:
df["varia_prem_anterior"] = df["precos_otimizados"]/df["premio_anterior"]
df["varia_prem_anterior_earnix"] = df["Premium Parameter"]/df["premio_anterior"]
df["varia_prem_ofer"] = df["precos_otimizados"]/df["premio"]
df["varia_prem_ofer_earnix"] = df["Premium Parameter"]/df["premio"]

In [ ]:
df[["varia_prem_anterior","varia_prem_ofer","varia_prem_anterior_earnix","varia_prem_ofer_earnix"]].describe().T

In [ ]:
ver = df.query('varia_prem_ofer >= 1.1')
ver

In [ ]:
#df_lote_otim_atu.groupby(["Amostra"])["Demanda_base"].describe().T

In [ ]:
ver[["premio_anterior","premio","precos_otimizados","Premium Parameter","LB","UB","varia_prem_anterior","varia_prem_ofer"]]

In [ ]:
#df[['premio','retencao_base_esperado','preco_otimizado','retencao_otimizada']]

In [ ]:
#df#.to_csv(r'C:/Users/vcatelan/Documents/Scripts/dftocsv2.csv', sep='|', encoding='utf-8', header='true')
#df.to_excel(r'df.xlsx', sep='|', encoding='utf-8', header='true')

#criando a função

In [ ]:
df_relatorio_otim = df.copy()

df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

In [ ]:
df.to_csv(r'df_lote-202410_teste3.csv', sep='|' ,header='true')

In [ ]:
df_relatorio_otim[['Demanda_base','retencao_otimizada','premio','preco_otimizado']]
, sep='|', encoding='utf-8', header='true'

In [ ]:
prices_df = pd.DataFrame({
    'chave': df_relatorio["chave"],
    'Act_cost':  df_relatorio["Act_cost"],
    'Premio ofertado com batente':  df_relatorio["Premio_oferecido_com_batente"],
    'Optimized Prices': df_relatorio["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_relatorio["prob_est"],
    'renewal demand':df_relatorio["demanda otimizada"],
    'variação':df_relatorio["premio"]/df_relatorio["Premio_oferecido_com_batente"]
})
prices_df.head(20)

########

In [ ]:
%%time
target_demand=0.47

# Função de demanda para calcular a retenção com base no preço 
def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(0.601-0.1513 * np.log(P)-1.9242 * np.divide(P, X1)-0.9005 * np.log(np.divide(P, X2))-0.9288 * np.log(np.divide(P, X3))-1.13 * np.log((np.divide(P, X4)))  + 0.2656 * np.log(X5 + 1)  + 0.011 * X6  + 0.1572 * X7  + 0.0683 * X8-0.2266 * X9  + 0.193 * X10-0.2988 * X11-0.2465 * X12-0.2347 * X13-0.0984 * X14-0.1547 * X15  + 0.175 * X16  + 0.299 * X17-0.4524 * X18-0.1465 * X19-0.7454 * X20  + 0.4683 * X21  + 1.1445 * X22  + 2.0254 * X23-0.2652 * X24-0.485*X25-0.43 * X26)
    return D/(1+D)

df['retencao_otimizada'] = calcular_retencao(precos_otimizados,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

df['retencao_target']=target_demand

fator_retencao = np.mean(df['Demanda_base'])/np.mean(df['retencao_target'])

# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(precos):
    df['precos']=precos
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada'] = calcular_retencao(precos,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
    
    df['margem_esperada_otimizada'] =   df['retencao_otimizada']*(df['precos'] - df['Act_cost'])
    
    # Calcular a margem esperada total (somatório)
    margem_esperada_total = df['margem_esperada_otimizada'].sum()

    # Maximizar a margem (retornamos o negativo para que o método minimize possa maximizar)
    return -margem_esperada_total

# Definir a restrição: a retenção média deve ser igual à retenção base média 
def restricao(precos):
    df['precos']=precos
    
    # Calcular a retenção otimizada média com os preços dados
    retencao_otimizada_soma = calcular_retencao(precos,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26).sum()
    
    return np.round(retencao_otimizada_soma,3) - np.round(df['retencao_target'].sum,3)

# Estimativas iniciais (os preços base)

#precos_iniciais = fator_retencao*df['premio'].values

premio_base = df['premio'].values

precos_iniciais = 1*df['premio'].values

# Definir os limites para os preços (0.9 * preco_base <= preco_otimizado <= 1.1 * preco_base) 
limites = [(0.9 * p, 1.1 * p) for p in premio_base]

# Definir as restrições no formato exigido pelo método 'minimize'
restricoes = ({'type': 'eq', 'fun': restricao})

# Executar a otimização 

solution = minimize(objetivo,
                    precos_iniciais,
                    method='L-BFGS-B',
                    bounds=limites) 
# Resultados
precos_otimizados = solution.x
margem_esperada_total_otimizada = -resultado.fun

# Verificar os preços otimizados e a margem esperada otimizada 
df['preco_otimizado'] = solution.x 
df['retencao_otimizada'] = calcular_retencao(precos_otimizados,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

df['margem_esperada_base_case'] = df['Demanda_base']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()

# Exibir os resultados
print(solution.message) 
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Retenção Média Base Case:", df['Demanda_base'].mean())
print("Premio Medio Otimizado:", df['preco_otimizado'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Tamanho da Base:", df.shape[0])


In [ ]:
#from handythread import foreach

In [ ]:
#!pip install pp